# 0. About This Colab Notebook

1. This Colab is designed for the course "**Computer Programming in Python**" instructed by Tse-Yu Lin.

2. Note that each time you enter this Colab from the link provided by the instructor, TAs or other people.
Please create a copy to your own Google Drive by clicking **File > Save a copy in Drive**. After clicking, a new tab page will be opened.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_palette(palette="pastel")
sns.set()


# a) Preparation: Download Dataset

We import the following package ``gdown`` to download files from Google Drive.

In [2]:
filename = "disney_plus_titles.csv"
if "get_ipython" in dir() and "google.colab" in str(get_ipython()):
    import gdown

    url = "https://drive.google.com/u/1/uc?id=118sEZB_-OfyXH130f-EED4KI26AXH8FO&export=download"
    # mirror_url = "https://drive.google.com/u/1/uc?id=1xlzqH8z2ChAO4xPpm3GDfslyni53Bn5j&export=download"
    gdown.download(url, filename)


In [3]:
raw_df = pd.read_csv(filename)


In [4]:
m, n = raw_df.shape
print(f"Size of dataset: {m}")
print(f"Number of Features: {n}")


Size of dataset: 1450
Number of Features: 12


In [5]:
raw_df.sample(5)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
436,s437,TV Show,Spaced Out,NaN,Ben DeTour,NaN,"August 14, 2020",2018,TV-G,1 Season,"Docuseries, Kids",Learn about our universe through imagery taken...
1145,s1146,Movie,Pooh's Heffalump Movie,Frank Nissen,"Jim Cummings, John Fiedler, Nikita Hopkins, Ka...",United States,"November 12, 2019",2005,G,73 min,"Action-Adventure, Animation, Kids",Pooh and his friends are shocked when Roo befr...
799,s800,TV Show,Coop & Cami Ask The World (Shorts),NaN,NaN,NaN,"November 12, 2019",2018,TV-G,1 Season,"Comedy, Family, Game Show / Competition","Cooper and Cami Wrather co-host ""Would You Wra..."
574,s575,Movie,Double Dribble,Jack Hannah,NaN,United States,"April 3, 2020",1946,TV-G,8 min,"Animation, Family, Sports",Goofy explores the wonderful world of basketba...
1358,s1359,TV Show,The World According to Jeff Goldblum,NaN,Jeff Goldblum,United States,"November 12, 2019",2019,TV-PG,2 Seasons,Docuseries,Jeff Goldblum explores the fascinating science...


Please refer the following link for more detail about headers:
* Disney+ Movies and TV Shows: https://www.kaggle.com/datasets/shivamb/disney-movies-and-tv-shows

# b) Data Analysis on Your Own

In [6]:
raw_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


## Clean the Data

In [20]:
cleaned_df = (
    raw_df.drop(labels=["director", "description"], axis="columns")
    .dropna(subset=["rating", "date_added"], how="any", axis="index")
    .assign(
        date_added=pd.to_datetime(raw_df["date_added"], format="%B %d, %Y"),
        duration=raw_df["duration"].str.extract("([0-9]+)").astype(int),
        cast=raw_df["cast"].str.split(", "),
        country=raw_df["country"].str.split(", "),
        listed_in=raw_df["listed_in"].str.split(", "),
    )
    .convert_dtypes()
    .set_index(keys=["type", "rating", "release_year", "date_added", "show_id"])
    .sort_index()
)

cleaned_df.info()
cleaned_df


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1444 entries, ('Movie', 'G', 1937, Timestamp('2019-11-12 00:00:00'), 's1200') to ('TV Show', 'TV-Y7-FV', 2021, Timestamp('2021-04-02 00:00:00'), 's266')
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      1444 non-null   string
 1   cast       1255 non-null   object
 2   country    1228 non-null   object
 3   duration   1444 non-null   Int32 
 4   listed_in  1444 non-null   object
dtypes: Int32(1), object(3), string(1)
memory usage: 114.6+ KB


title  \
type    rating   release_year date_added show_id                                                   
Movie   G        1937         2019-11-12 s1200                   Snow White and The Seven Dwarfs   
                 1940         2019-11-12 s1130                                         Pinocchio   
                                         s884                                           Fantasia   
                 1941         2019-11-12 s1326                              The Reluctant Dragon   
                 1942         2019-11-12 s739                                              Bambi   
...                                                                                          ...   
TV Show TV-Y7-FV 1985         2021-06-18 s196                          Star Wars Vintage: Droids   
                 2015         2019-11-12 s1005         LEGO Star Wars: Resistance Rises (Shorts)   
                 2019         2019-11-12 s1214                                  Star Wars: Blips   
                 2020         2021-02-12 s301     Marvel Battleworld: Mystery of the Thanostones   
                 2021         2021-04-02 s266      Star Wars Vintage: Clone Wars 2D Micro-Series   

                                                                                               cast  \
type    rating   release_year date_added show_id                                                      
Movie   G        1937         2019-11-12 s1200    [Adriana Caselotti, Harry Stockwell, Lucille L...   
                 1940         2019-11-12 s1130    [Dickie Jones, Christian Rub, Cliff Edwards, E...   
                                         s884        [Deems Taylor, Walt Disney, Leopold Stokowski]   
                 1941         2019-11-12 s1326    [Robert Benchley, Gerald Mohr, Alan Ladd, Flor...   
                 1942         2019-11-12 s739                                                   NaN   
...                                                                                             ...   
TV Show TV-Y7-FV 1985         2021-06-18 s196     [Anthony Daniels, R2-D2, Lesleh Donaldson, And...   
                 2015         2019-11-12 s1005                                                  NaN   
                 2019         2019-11-12 s1214                                                  NaN   
                 2020         2021-02-12 s301     [Giles Panton, Rebecca Shoichet, Jesse Inocall...   
                 2021         2021-04-02 s266     [Mat Lucas, James Arnold Taylor, Tom Kane, Gre...   

                                                                   country  \
type    rating   release_year date_added show_id                             
Movie   G        1937         2019-11-12 s1200             [United States]   
                 1940         2019-11-12 s1130             [United States]   
                                         s884              [United States]   
                 1941         2019-11-12 s1326             [United States]   
                 1942         2019-11-12 s739              [United States]   
...                                                                    ...   
TV Show TV-Y7-FV 1985         2021-06-18 s196                          NaN   
                 2015         2019-11-12 s1005    [United States, Denmark]   
                 2019         2019-11-12 s1214             [United States]   
                 2020         2021-02-12 s301              [United States]   
                 2021         2021-04-02 s266                          NaN   

                                                  duration  \
type    rating   release_year date_added show_id             
Movie   G        1937         2019-11-12 s1200          84   
                 1940         2019-11-12 s1130          90   
                                         s884          126   
                 1941         2019-11-12 s1326          74   
                 1942         2019-11-12 s739           72   
.